In [ ]:
'''
Script in which the resultant best HP's from optuna runs are used for model architectures, they are then trained and evaluated.

res: dict containing for each loss function and each model, the resultant metrics from evaluation.
'''

In [1]:
import sys
sys.path.append("../")

In [ ]:
import os
import json
from src.utils_ import load_json, set_seed
import pandas as pd
from src.data_ import DataPrep

set_seed(42)

/home/bengawith/workspaces/github.com/bengawith/project/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dp = DataPrep('../data/csv/14KP_48CLA.csv')
data = dp.get_data()

en_d = {
    'Standard_GRUModel_top_trials.json': 'GRUModel',
    'Standard_LSTMModel_top_trials.json': 'LSTMModel',
    'Standard_FullyConnectedNN_top_trials.json': 'FullyConnectedNN',
    'Standard_CNNModel_top_trials.json': 'CNNModel',
    'FeatureBinning_GRUModel_top_trials.json': 'GRUModel',
    'FeatureBinning_LSTMModel_top_trials.json': 'LSTMModel',
    'FeatureBinning_FullyConnectedNN_top_trials.json': 'FullyConnectedNN',
    'FeatureBinning_CNNModel_top_trials.json': 'CNNModel',
}

In [7]:
res = json.load(open('./final/json/final_res.json', 'r'))

In [ ]:
from model_tune_anal import process_file_2

res = {}

res, results = process_file_2(data, res)

In [7]:
old_res = json.load(open('./final/json/final_res.json', 'r'))
old_results = json.load(open('./final/json/final_results.json', 'r'))

In [ ]:
for lf in results:
    for net in results[lf]:
        for metric in results[lf][net]:
            if metric == 'r2' and results[lf][net][metric] > old_results[lf][net][metric]:
                print('results', lf, net, metric, results[lf][net][metric])
                print('old_results', lf, net, metric, old_results[lf][net][metric])

In [14]:
temp = {
    "res": old_res,
    "results": old_results
}

In [16]:
for k, v in temp.items():
    with open(f'./final/json/{k}.json', 'w') as f:
        json.dump(v, f, indent=4)

In [26]:
top_dict = {
    'pi': {
        'r2':{'score': 0, 'net': None},
        'mse': {'score': float('inf'), 'net': None},
        'rmse': {'score': float('inf'), 'net': None},
        'mae': {'score': float('inf'), 'net': None,}
    },    
    'pi_new': {
        'r2':{'score': 0, 'net': None},
        'mse': {'score': float('inf'), 'net': None},
        'rmse': {'score': float('inf'), 'net': None},
        'mae': {'score': float('inf'), 'net': None,}
    },
    'huber': {
        'r2':{'score': 0, 'net': None},
        'mse': {'score': float('inf'), 'net': None},
        'rmse': {'score': float('inf'), 'net': None},
        'mae': {'score': float('inf'), 'net': None,}
    },
    'mse': {
        'r2':{'score': 0, 'net': None},
        'mse': {'score': float('inf'), 'net': None},
        'rmse': {'score': float('inf'), 'net': None},
        'mae': {'score': float('inf'), 'net': None,}
    },
    'log_cosh': {
        'r2':{'score': 0, 'net': None},
        'mse': {'score': float('inf'), 'net': None},
        'rmse': {'score': float('inf'), 'net': None},
        'mae': {'score': float('inf'), 'net': None,}
    }
}

for lf in results:
    for net in results[lf]:
        for metric in results[lf][net]:
            if metric != 'r2':
                if results[lf][net][metric] < top_dict[lf][metric]['score']:
                    top_dict[lf][metric]['score'] = results[lf][net][metric]
                    top_dict[lf][metric]['net'] = net
            else:
                if results[lf][net][metric] > top_dict[lf][metric]['score']:
                    top_dict[lf][metric]['score'] = results[lf][net][metric]
                    top_dict[lf][metric]['net'] = net


In [ ]:
top_dict

In [28]:
results['top_dict'] = top_dict

In [3]:
import json

results = json.load(open('./final/json/final_results.json'))

In [17]:
pi_res = old_results['pi']
huber_res = old_results['huber']
mse_res = old_results['mse']
log_cosh_res = old_results['log_cosh']

In [31]:
top_res = results['top_dict']

In [32]:
top_res_df = pd.DataFrame.from_dict(top_res, orient='index')

In [ ]:
top_res_df

In [ ]:
rows = []
for method, metrics in top_res.items():
    for metric, values in metrics.items():
        rows.append({
            'Loss Func': method,
            'Metric': metric,
            'Score': values['score'],
            'Model': values['net']
        })

In [ ]:
top_dict_df = pd.DataFrame(rows)

In [37]:
os.makedirs('csv/final', exist_ok=True)
top_dict_df.to_csv('csv/final/top_results.csv')

In [38]:
with open('final_results.json', 'w') as f:
    json.dump(results, f, indent=4)

In [18]:
import pandas as pd
pi_res_df = pd.DataFrame.from_dict(pi_res, orient='index').T
huber_res_df = pd.DataFrame.from_dict(huber_res, orient='index').T
mse_res_df = pd.DataFrame.from_dict(mse_res, orient='index').T
log_cosh_res_df = pd.DataFrame.from_dict(log_cosh_res, orient='index').T

In [ ]:
for name, res_df in {'pi': pi_res_df, 'huber': huber_res_df, 'mse': mse_res_df, 'log_cosh': log_cosh_res_df}.items():
    res_df.to_csv(f'optuna_results/csv/{name.split("_")[0]}_results.csv')